In [1]:
import pandas as pd
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import train_test_split, cross_validate
from rs_datasets import MovieLens
import pickle

from sim4rec.utils import pandas_to_spark
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName('simulator_movielens')\
    .master('local[*]')\
    .config('spark.sql.shuffle.partitions', '192')\
    .config('spark.default.parallelism', '192')\
    .config('spark.driver.extraJavaOptions', '-XX:+UseG1GC')\
    .config('spark.executor.extraJavaOptions', '-XX:+UseG1GC')\
    .config('spark.driver.memory', '256g')\
    .getOrCreate()

data = spark.read.parquet('train_trim.parquet')

24/05/26 18:42:55 WARN Utils: Your hostname, ecs-syudosaev-big resolves to a loopback address: 127.0.1.1; using 10.11.12.124 instead (on interface eth0)
24/05/26 18:42:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/26 18:42:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
als = ALS(userCol="user_id",
          itemCol="song_id",
          ratingCol="rating",
          implicitPrefs=True,
          rank=128,
          maxIter=15,
          seed=12345)
model = als.fit(data)

24/05/26 18:43:20 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/26 18:43:20 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
24/05/26 18:43:21 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


24/05/26 19:00:28 WARN DAGScheduler: Broadcasting large task binary with size 1064.9 KiB


24/05/26 19:01:48 WARN DAGScheduler: Broadcasting large task binary with size 1132.8 KiB


24/05/26 19:03:06 WARN DAGScheduler: Broadcasting large task binary with size 1065.8 KiB
24/05/26 19:03:06 WARN DAGScheduler: Broadcasting large task binary with size 1200.8 KiB


24/05/26 19:04:25 WARN DAGScheduler: Broadcasting large task binary with size 1133.7 KiB
24/05/26 19:04:25 WARN DAGScheduler: Broadcasting large task binary with size 1268.7 KiB


...

24/05/26 19:17:37 WARN DAGScheduler: Broadcasting large task binary with size 1813.1 KiB
24/05/26 19:17:37 WARN DAGScheduler: Broadcasting large task binary with size 1948.1 KiB


24/05/26 19:18:56 WARN DAGScheduler: Broadcasting large task binary with size 1881.0 KiB
24/05/26 19:18:56 WARN DAGScheduler: Broadcasting large task binary with size 2016.0 KiB


24/05/26 19:20:15 WARN DAGScheduler: Broadcasting large task binary with size 1949.0 KiB
24/05/26 19:20:16 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB


24/05/26 19:21:34 WARN DAGScheduler: Broadcasting large task binary with size 2016.9 KiB
24/05/26 19:21:34 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB


24/05/26 19:22:54 WARN DAGScheduler: Broadcasting large task binary with size 2017.5 KiB


In [3]:
user_embs = model.userFactors.toPandas()
user_cols = [f'uembed_{i}' for i in range(128)]
user_features = pd.DataFrame(user_embs['features'].tolist(), columns=user_cols)
user_features['user_id'] = user_embs['id']

item_embs = model.itemFactors.toPandas()
item_cols = [f'iembed_{i}' for i in range(128)]
item_features = pd.DataFrame(item_embs['features'].tolist(), columns=item_cols)
item_features['song_id'] = item_embs['id']

24/05/26 19:26:33 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
24/05/26 19:26:39 WARN DAGScheduler: Broadcasting large task binary with size 2024.6 KiB


In [4]:
user_features.to_parquet('user_svd.parquet', index=False)
item_features.to_parquet('item_svd.parquet', index=False)